In [54]:
!pip install requests-html selenium tdqm


Defaulting to user installation because normal site-packages is not writeable


In [1]:
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from time import sleep
from selenium.webdriver.common.by import By
from tqdm import tqdm


# create an HTML Session object
session = HTMLSession()

# base url
url = 'https://www.wcostream.org/anime/fire-force'

# get the html

r = session.get(url)

# create the soup object
html = r.html

# find all the episodes
episodes = html.find('#catlist-listview li a')

urls = []
for episode in episodes:
    url = episode.attrs['href']
    urls.append(url)
print(urls)


['https://www.wcostream.org/fire-force-season-2-episode-24-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-23-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-22-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-21-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-20-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-19-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-18-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-17-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-16-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-15-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-14-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-13-english-dubbed', 'https://www.wcostream.org/fire-force-season-2-episode-12-english-dubbed', 'https://www.wcostream.o

In [3]:
firefox_options = Options()
firefox_options.add_argument('--disable-gpu')
firefox_options.add_argument('--headless')
service = Service('/snap/bin/firefox.geckodriver')

driver = webdriver.Firefox(service=service, options=firefox_options)

for detailurl in urls[5:21]:
    driver.get(detailurl)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(3)
    iframe = driver.find_element(By.ID, 'frameNewcizgifilmuploads0')
    if iframe is None:
        print('No iframe found in the HTML')
        continue
    iframe_url = iframe.get_attribute('src')
  
    # switch to the iframe window
    driver.switch_to.frame(iframe)
    sleep(3)
    video = driver.find_element(By.TAG_NAME, 'video')
    if video is None:
        print('No video found in the HTML')
        continue
    video_url = video.get_attribute('src')
    print(video_url)

    # https://www.wcostream.org/fire-force-season-2-episode-22-english-dubbed
    filename = detailurl.split('/')[-1] + '.mp4'
    print(filename)
    header = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Range': 'bytes=0-',
        'Referer': detailurl,
        'Origin': 'https://www.wcostream.org',
        'Connection': 'keep-alive',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache',
        # Requests doesn't support trailers
        # 'TE': 'trailers',
    }

    r = session.get(video_url, headers=header, stream=True)
    total_size = int(r.headers['Content-Length']) 
    
    with open(filename, 'wb') as f:
        progress_bar = tqdm(total=total_size, unit='B', unit_scale=True) 
        for chunk in r.iter_content(chunk_size=1024):
            f.write(chunk)
            progress_bar.update(len(chunk))

/tmp/ipykernel_198660/1090904147.py:4: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service('/snap/bin/firefox.geckodriver')
 17%|██████▏                             | 16.4M/94.7M [08:40<1:48:20, 12.0kB/s]

KeyboardInterrupt: 

In [7]:
driver.quit()

In [ ]:
from threading import Thread

def download_video(video_url, filename, session):
    header = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Range': 'bytes=0-',
        'Referer': detailurl,
        'Origin': 'https://www.wcostream.org',
        'Connection': 'keep-alive',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache',
        # Requests doesn't support trailers
        # 'TE': 'trailers',
    }

    r = session.get(video_url, headers=header, stream=True)
    total_size = int(r.headers['Content-Length']) 
    
    with open(filename, 'wb') as f:
        progress_bar = tqdm(total=total_size, unit='B', unit_scale=True) 
        for chunk in r.iter_content(chunk_size=1024):
            f.write(chunk)
            progress_bar.update(len(chunk))

threads = []

firefox_options = Options()
firefox_options.add_argument('--disable-gpu')
#firefox_options.add_argument('--headless')
service = Service('/snap/bin/firefox.geckodriver')

driver = webdriver.Firefox(service=service, options=firefox_options)

for detailurl in urls[5:24]:
    driver.get(detailurl)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(3)
    iframe = driver.find_element(By.ID, 'frameNewcizgifilmuploads0')
    if iframe is None:
        print('No iframe found in the HTML')
        continue
    iframe_url = iframe.get_attribute('src')
  
    # switch to the iframe window
    driver.switch_to.frame(iframe)
    sleep(3)
    video = driver.find_element(By.TAG_NAME, 'video')
    if video is None:
        print('No video found in the HTML')
        continue
    video_url = video.get_attribute('src')
    print(video_url)

    # https://www.wcostream.org/fire-force-season-2-episode-22-english-dubbed
    filename = detailurl.split('/')[-1] + '.mp4'
    print(filename)

    t = Thread(target=download_video, args=(video_url, filename, session))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

/tmp/ipykernel_198660/1921453022.py:32: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service('/snap/bin/firefox.geckodriver')


https://ndisk.cizgifilmlerizle.com/getvid?evid=qNdDoYpe_T6BKCzw5BpUvSnZ7Efhw0Zx-pppBmE8CF3cInH5EiGdTCnx7ZpTMgsalbYZ3VmDuvlCy8U_qpcVVCk8MykU6CilALhzTzi55k-mmwAkyRUGrtHsAiELL-Iyao_Tdt1ywSD8MEPuCj553kUihLPZGTVhjF8X3yt0-Ez3CSGcMft3o4ejkTH2xETdb2vc_i8ORUeKkzlYkrn7Om8tBTQBfvjSL1PoyMHxt1ZM23pLmTnOTjuYHIzgQ87dafKn47GPJupWejG9Y5RZMciOMTHJrV2337B-cg2oyTAqHGHRTNMFExRrU7dKhL0_mNCU-WhVgMsFCekmcaytR_ubeC9dZD85if6lCXo5aaanKm4jX570S_CI09ErvQjq
fire-force-season-2-episode-19-english-dubbed.mp4



  0%|                                               | 0.00/94.7M [00:00<?, ?B/s]
  0%|                                    | 16.4k/94.7M [00:01<1:41:45, 15.5kB/s]
  0%|                                    | 32.8k/94.7M [00:01<1:18:34, 20.1kB/s]

https://ndisk.cizgifilmlerizle.com/getvid?evid=qNdDoYpe_T6BKCzw5BpUvSnZ7Efhw0Zx-pppBmE8CF3cInH5EiGdTCnx7ZpTMgsalbYZ3VmDuvlCy8U_qpcVVOkFZMJgTIyW-xMpjSW8iQj7d2oZE6SaCaR-6TXk6qS9-Sw08NuIG64uYqrfrQQ1pldM9dHzVo8-xILiJbRqRb43r_VAELHA49nN_wqtRpaqPqLWLI7-pp9C1Jhrq5CIDceK7dqpmMcFZVYJn0FF3M_dLMc0zpmfVow0TOa6iZUrt60XtMug7z-29vdgnAqzZ8LMHPMcYN4Yfk6FrAYaBOLjB7vsTgBPC3JwdFqjeh_tT3LJz8ro28OnqHjU92aHYWFNKMPA2kXqcJ8CdgbjdvNPxjA39qjzMT0WNKP3L6kX
fire-force-season-2-episode-18-english-dubbed.mp4



  0%|                                    | 49.2k/94.7M [00:02<1:20:02, 19.7kB/s]
  0%|                                    | 65.5k/94.7M [00:02<1:04:15, 24.5kB/s]
  0%|                                    | 81.9k/94.7M [00:04<1:18:29, 20.1kB/s]
  0%|                                    | 98.3k/94.7M [00:04<1:17:18, 20.4kB/s]
  0%|                                     | 115k/94.7M [00:05<1:17:31, 20.3kB/s]
  0%|                                     | 131k/94.7M [00:06<1:08:41, 22.9kB/s]
  0%|                                     | 147k/94.7M [00:07<1:22:08, 19.2kB/s]
  0%|                                     | 164k/94.7M [00:08<1:20:21, 19.6kB/s]
  0%|                                     | 180k/94.7M [00:08<1:06:02, 23.9kB/s]
  0%|                                     | 197k/94.7M [00:08<1:01:11, 25.7kB/s]

  0%|                                                | 0.00/112M [00:00<?, ?B/s]

  0%|                                       | 32.8k/112M [00:00<20:05, 92.9kB/s]
  0%|                    

...











  3%|█▎                                     | 4.80M/140M [00:54<58:01, 38.7kB/s]










  1%|▍                                    | 1.80M/160M [00:43<1:30:05, 29.2kB/s]






  1%|▎                                     | 934k/106M [01:16<5:34:01, 5.26kB/s]

 13%|█████                                  | 14.6M/112M [02:02<38:01, 42.7kB/s]







  5%|█▊                                     | 5.62M/121M [01:07<28:09, 68.2kB/s]





  1%|▎                                     | 868k/107M [01:25<3:01:11, 9.77kB/s]




  9%|███▎                                   | 9.52M/111M [01:41<41:02, 41.2kB/s]









  3%|█▎                                     | 4.82M/140M [00:54<48:23, 46.4kB/s]








  0%|▏                                     | 541k/111M [00:58<3:26:05, 8.92kB/s]







  5%|█▊                                     | 5.64M/121M [01:07<24:41, 77.8kB/s]

 13%|█████                                  | 14.6M/112M [02:02<24:52, 65.3kB/s]










  1%|▍                         

https://ndisk.cizgifilmlerizle.com/getvid?evid=z1wcyGBv6unGsPkB34OY1wkJE25VVae2UzAKjc5cBdbmwK9vso5WGV5ZLLMqTcIAbWUPffh_Ww-DIuuv1-Xv9yE8abcN2sWrn_2WJGTBk7gztPHB8dlfaB7-2p4J1765n7unZ-zf-bf5f7mF4HqYC_0FQ13XlIzacs0NK7UfXWIchdZO08on60fcpFEhko5RAZ-5vcZvErI5v12DBf84CZv6cp0WrtX4XkuKvQxWEBOHPhpctrWdTrihoPxMN579oCSa1TfMI3Wd8cM1N6v7R_JXVME8A99wuaxRTyIObF4cHL_GRvlHDTsTb444zZF1I4CeD0YOmHucujejuF2r2g
fire-force-season-2-episode-5-english-dubbed.mp4







  9%|███▍                                   | 9.95M/111M [01:49<37:17, 45.1kB/s]







  5%|██▏                                     | 6.64M/121M [01:15<14:08, 135kB/s]










  2%|▋                                       | 2.84M/160M [00:50<09:40, 270kB/s]










  2%|▋                                       | 2.88M/160M [00:51<08:45, 298kB/s]









  4%|█▎                                   | 5.16M/140M [01:02<1:12:23, 30.9kB/s]








  1%|▏                                     | 639k/111M [01:06<3:10:00, 9.66kB/s]













  0%|                                      | 164k/108M [00:06<1:43:53, 17.3kB/s]




  9%|███▌                                   | 9.96M/111M [01:49<33:42, 49.9kB/s]





  1%|▎                                     | 967k/107M [01:33<3:27:53, 8.51kB/s]







  6%|██▏                                     | 6.68M/121M [01:15<13:25, 142kB/s]

 14%|█████▍                                 | 15.5M/112M [02:10<21:21, 75.4kB/s]










  2%|▋              

https://ndisk.cizgifilmlerizle.com/getvid?evid=z1wcyGBv6unGsPkB34OY19gXF7OHYN-Q5OOfIMupiYlHyzaS9PBQHQXlzOFWaKVbTGOKpodAmIiaBcH6UZNFeIWTA3ZuXOZXXOx6v5em7vr6e_CfDCeqBTh6dyMwRm3O-lMG5zu8_0ZJd55ODJ9ZBvDecYg_hj1aPF0NSycKHaZiHzHmcV-vZavyNGQnCLvqBjj8S_mtRNApPKbecwkArjpJE_8YRlOn-n_MmKEXB-CzoHmsQ1eXDEyaAESvlMF0UAyo3HkDoBf9H18Yagih1rp9CY54zRJgi6hpO20QLCc
fire-force-season-2-episode-4-english-dubbed.mp4




 14%|█████▊                                  | 16.3M/112M [02:19<11:12, 142kB/s]










  3%|█                                      | 4.19M/160M [00:59<51:38, 50.2kB/s]







  6%|██▏                                  | 7.32M/121M [01:24<1:00:03, 31.5kB/s]














  1%|▎                                        | 901k/130M [00:06<18:49, 114kB/s]




  9%|███▍                                 | 10.3M/111M [01:58<1:20:24, 20.9kB/s]














  1%|▎                                        | 950k/130M [00:06<16:38, 129kB/s]









  4%|█▍                                   | 5.52M/140M [01:11<1:51:12, 20.1kB/s]

 15%|█████▋                                 | 16.3M/112M [02:19<17:15, 92.6kB/s]











  0%|                                      | 147k/129M [00:36<9:16:33, 3.85kB/s]





  1%|▎                                    | 1.05M/107M [01:42<3:36:48, 8.15kB/s]










  3%|█                                      | 4.26M/160M [01:00<38:26, 67.4kB/s]









  4%|█▍      